In [1]:
import pandas as pd
import numpy as np
import mpu
import seaborn as sns
import re

In [2]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 100)

In [3]:
import geopandas as gpd
import geopy
from shapely.geometry import Point, Polygon
from geopy.geocoders import Nominatim

/home/abduallah/.local/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [6]:
geo_df = gpd.read_file('Reading shape files/egy_admbnda_adm3_capmas_20170421.shp')
geo_df.head()

,ADM3_EN,ADM3_AR,ADM3_PCODE,ADM3_REF,ADM3ALT1EN,ADM3ALT2EN,ADM3ALT1AR,ADM3ALT2AR,ADM2_EN,ADM2_AR,ADM2_PCODE,ADM1_EN,ADM1_AR,ADM1_PCODE,ADM0_EN,ADM0_AR,ADM0_PCODE,date,validOn,validTo,Shape_Leng,Shape_Area,geometry
0,None,الجزيره,EG120918,None,None,None,None,None,Dikirnis,مركز دكرنس,EG1209,Dakahlia,الدقهلية,EG12,Egypt,مِصر,EG,2006-01-01,2017-04-21,None,0.055274,0.000157,"POLYGON ((31.54448 31.08392, 31.54505 31.07872..."
1,None,الجهاد 4,EG240525,None,None,None,None,None,Markz Al Idwa,مركز العدوة,EG2405,Menia,المنيا\n,EG24,Egypt,مِصر,EG,2006-01-01,2017-04-21,None,0.101889,0.000594,"POLYGON ((30.73757 28.74247, 30.73682 28.74252..."
2,None,أم الرضا الجديدة,EG110437,None,None,None,None,None,Kafr Sad,مركز كفر سعد,EG1104,Damietta,دمياط\n,EG11,Egypt,مِصر,EG,2006-01-01,2017-04-21,None,0.049183,0.000103,"POLYGON ((31.63090 31.41011, 31.62543 31.40981..."
3,None,رأس محمد,EG350502,None,None,None,None,None,Sharm el-Sheikh,قسم شرم الشيخ,EG3505,South Sinai,جنوب سيناء,EG35,Egypt,مِصر,EG,2006-01-01,2017-04-21,None,0.044184,0.000094,"POLYGON ((34.24396 27.75904, 34.24485 27.74441..."
4,None,مبارك,EG120565,None,None,None,None,None,Sinbillawin,مركز السنبلاوين,EG1205,Dakahlia,الدقهلية,EG12,Egypt,مِصر,EG,2006-01-01,2017-04-21,None,0.044160,0.000122,"POLYGON ((31.36003 30.86623, 31.36525 30.86547..."


In [5]:
# cleaning the shapefile
all_sub_regions = geo_df[['ADM3_AR','ADM2_AR','ADM1_AR']]

all_sub_regions['ADM1_AR'] = all_sub_regions['ADM1_AR'].apply(lambda x : re.sub('\n','',x))
all_sub_regions['ADM2_AR'] = all_sub_regions['ADM2_AR'].apply(lambda x : re.sub('\n','',x))
all_sub_regions['ADM3_AR'] = all_sub_regions['ADM3_AR'].apply(lambda x : re.sub('\n','',x))
all_sub_regions.rename(columns={
    'ADM3_AR':'منطقة',
    'ADM2_AR':'قسم / مركز',
    'ADM1_AR':'محافظة',
},inplace=True)
all_sub_regions

/tmp/ipykernel_2079387/2496682489.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_sub_regions['ADM1_AR'] = all_sub_regions['ADM1_AR'].apply(lambda x : re.sub('\n','',x))
/tmp/ipykernel_2079387/2496682489.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_sub_regions['ADM2_AR'] = all_sub_regions['ADM2_AR'].apply(lambda x : re.sub('\n','',x))
/tmp/ipykernel_2079387/2496682489.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

,منطقة,قسم / مركز,محافظة
0,الجزيره,مركز دكرنس,الدقهلية
1,الجهاد 4,مركز العدوة,المنيا
2,أم الرضا الجديدة,مركز كفر سعد,دمياط
3,رأس محمد,قسم شرم الشيخ,جنوب سيناء
4,مبارك,مركز السنبلاوين,الدقهلية
...,...,...,...
5711,ونا القس,مركز الواسطى,بنى سويف
5712,ونينة الشرقية,مركز سوهاج,سوهاج
5713,ونينة الغربية,مركز سوهاج,سوهاج
5714,يعقوب,مركز البلينا,سوهاج


In [28]:
atm_df = pd.read_excel('ATM_Bank_Misr_except_in_branches.xlsx')
atm_df.head()

,ID,Address,منطقة,قسم / مركز,محافظة,Lat,Long,Nearest_Branch_distance,Nearest_Branch_Address,Lat_branch,Long_branch,منطقة الفرع,قسم / مركز الفرع,محافظة الفرع
0,1,Fayoum University,قسم ثانى,قسم الفيوم,الفيوم,29.319696,30.835641,1480.524070,Fayoum Security Directorate building,29.310533,30.84672,قسم ثانى,قسم الفيوم,الفيوم
1,2,Fayoum AlHoriea St.,قسم اول,قسم الفيوم,الفيوم,29.308199,30.842229,382.882978,AlFayoum 45 AlHorreya St. Fayoum,29.307723,30.84614,قسم اول,قسم الفيوم,الفيوم
2,3,10th of Ramadan - Industrial Zone B1,قسم ثانى,قسم الفيوم,الفيوم,29.321487,30.837896,1488.462407,Fayoum Security Directorate building,29.310533,30.84672,قسم ثانى,قسم الفيوم,الفيوم
3,4,Faculty of Arts Fayoum University,قسم ثانى,قسم الفيوم,الفيوم,29.322145,30.837087,1593.570183,Fayoum Security Directorate building,29.310533,30.84672,قسم ثانى,قسم الفيوم,الفيوم
4,5,Fayoum train station,قسم اول,قسم الفيوم,الفيوم,29.308889,30.847499,184.899213,AlFayoum 45 AlHorreya St. Fayoum,29.307723,30.84614,قسم اول,قسم الفيوم,الفيوم


In [29]:
temp = atm_df
temp2 = all_sub_regions

In [30]:
# count atms in all subregions
def get_atms_counts_subregions(x):
    index = x['منطقة']
    count =0
    for i in range(len(temp)):
        if temp['منطقة'].iloc[i] == index:
            count += 1
    
    x['Num_of_Atms'] = count
    return x
    

In [31]:
all_sub_regions = temp2.apply(get_atms_counts_subregions,axis = 1)
all_sub_regions

,منطقة,قسم / مركز,محافظة,Num_of_Atms
0,الجزيره,مركز دكرنس,الدقهلية,0
1,الجهاد 4,مركز العدوة,المنيا,0
2,أم الرضا الجديدة,مركز كفر سعد,دمياط,0
3,رأس محمد,قسم شرم الشيخ,جنوب سيناء,0
4,مبارك,مركز السنبلاوين,الدقهلية,0
...,...,...,...,...
5711,ونا القس,مركز الواسطى,بنى سويف,0
5712,ونينة الشرقية,مركز سوهاج,سوهاج,0
5713,ونينة الغربية,مركز سوهاج,سوهاج,0
5714,يعقوب,مركز البلينا,سوهاج,0


In [33]:
all_sub_regions = all_sub_regions.sort_values(by='Num_of_Atms',ascending=False)
all_sub_regions
# clean text "\n" for all text 
# try to see why some places doesn't have atms 

,منطقة,قسم / مركز,محافظة,Num_of_Atms
3477,مدينة الشيخ زايد,قسم الشيخ زايد,الجيزة,76
1769,القرى السياحية,قسم أول الغردقة,البحر الأحمر,64
4879,مدينة شرم الشيخ وتشمل منطق,قسم شرم الشيخ,جنوب سيناء,54
2922,خارج زمام الشياخات,قسم أول 6 أكتوبر,الجيزة,53
2923,خارج زمام الشياخات,قسم ثان 6 أكتوبر,الجيزة,53
...,...,...,...,...
2109,المنشأة الجديدة,مركز السنطة,الغربية,0
2108,المنشأة الجديدة,مركز كفر صقر,الشرقية,0
2107,المنشأة الصغري,مركز كفر شكر,القليوبية,0
2105,المنسي,قسم باب الشعرية,القاهرة,0


In [34]:
all_sub_regions.to_excel('Num_of_Atms_Subregions.xlsx',index=False)

In [42]:
# df = pd.read_excel('Num_of_Atms_Subregions.xlsx')

In [46]:
geo_df = gpd.read_file('Reading shape files/egy_admbnda_adm1_capmas_20170421.shp')
geo_df.head()

,ADM1_EN,ADM1_AR,ADM1_PCODE,ADM1_REF,ADM1ALT1EN,ADM1ALT2EN,ADM1ALT1AR,ADM1ALT2AR,ADM0_EN,ADM0_AR,ADM0_PCODE,date,validOn,validTo,Shape_Leng,Shape_Area,geometry
0,Alexandria,الاسكندرية\n,EG02,Alexandria,None,None,None,None,Egypt,مِصر,EG,2006-01-01,2017-04-21,None,4.266566,0.236661,"POLYGON ((30.08063 31.33118, 30.08057 31.33114..."
1,Assiut,أسيوط,EG25,Assiut,None,None,None,None,Egypt,مِصر,EG,2006-01-01,2017-04-21,None,8.004909,1.498030,"MULTIPOLYGON (((30.75499 27.61106, 30.75618 27..."
2,Aswan,أسوان,EG28,Aswan,None,None,None,None,Egypt,مِصر,EG,2006-01-01,2017-04-21,None,38.126683,5.102297,"MULTIPOLYGON (((32.87266 24.17076, 32.87238 24..."
3,Behera,البحيرة\n,EG18,Behera,None,None,None,None,Egypt,مِصر,EG,2006-01-01,2017-04-21,None,6.508432,1.067535,"POLYGON ((30.39156 31.44026, 30.39281 31.44023..."
4,Beni Suef,بنى سويف\n,EG22,Beni Suef,None,None,None,None,Egypt,مِصر,EG,2006-01-01,2017-04-21,None,7.437239,0.981067,"MULTIPOLYGON (((30.96379 28.89630, 30.96116 28..."


In [47]:
all_governorate = geo_df[['ADM1_AR']]

all_governorate['ADM1_AR'] = all_governorate['ADM1_AR'].apply(lambda x : re.sub('\n','',x))
all_governorate.rename(columns={
    'ADM1_AR':'محافظة',
},inplace=True)
all_governorate

/tmp/ipykernel_1075117/3968291548.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_governorate['ADM1_AR'] = all_governorate['ADM1_AR'].apply(lambda x : re.sub('\n','',x))
/tmp/ipykernel_1075117/3968291548.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_governorate.rename(columns={


,محافظة
0,الاسكندرية
1,أسيوط
2,أسوان
3,البحيرة
4,بنى سويف
5,القاهرة
6,الدقهلية
7,دمياط
8,الفيوم
9,الغربية


In [48]:
temp = atm_df
temp2 = all_governorate

In [49]:
def get_atms_counts_governorate(x):
    index = x['محافظة']
    count =0
    for i in range(len(temp)):
        if temp['محافظة'].iloc[i] == index:
            count += 1
    
    x['Num_of_Atms'] = count
    return x
    

In [50]:
all_governorate = temp2.apply(get_atms_counts_governorate,axis = 1)
all_governorate

,محافظة,Num_of_Atms
0,الاسكندرية,251
1,أسيوط,56
2,أسوان,89
3,البحيرة,145
4,بنى سويف,61
5,القاهرة,834
6,الدقهلية,225
7,دمياط,35
8,الفيوم,30
9,الغربية,116


In [51]:
all_governorate.sort_values(['Num_of_Atms'],ascending= 0)

,محافظة,Num_of_Atms
5,القاهرة,834
10,الجيزة,454
0,الاسكندرية,251
6,الدقهلية,225
23,الشرقية,159
3,البحيرة,145
13,القليوبية,133
22,البحر الأحمر,122
26,سوهاج,120
17,المنوفية,116


In [52]:
all_governorate.to_excel('Num_of_Atms_Governorate.xlsx',index=False)